In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

## Preprocessing

In [2]:
## set parameters
#ROOT = 'PATH/TO/data/processed/'
ROOT = '/home/khlee/git/recommendation/GRU4Rec_TensorFlow/data/processed/'
DATA_TYPE = 'sample'
PATH_TO_TRAIN = ROOT + 'rsc15_train_{}.txt'.format(DATA_TYPE)
PATH_TO_TEST = ROOT + 'rsc15_test_{}.txt'.format(DATA_TYPE)
checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)
        
layers = 1
rnn_size = 100
batch_size = 50
drop_keep_prob = 0.7

n_epochs = 3
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
grad_cap = 0
print_step = 1e3

In [3]:
## load data
data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [4]:
## check sort data
### preprocessing에서 session, timestamp로 sorting을 하였는데,
### sorting이 중요하기 때문에 한번더 확인해본다.
### data가 클경우 sort에서 시간이 오래 걸리기 때문에 sample check을 수행한다.
def check_data_sort(dt, sample_check=False, sample_size=10000):
    if sample_check:
        sess_ids = dt['SessionId'].unique()
        sample_sess_ids = np.random.choice(sess_ids, sample_size, replace=False)
        dt = dt[np.in1d(dt.SessionId, sample_sess_ids)]
    ordered_dt = dt.sort_values(['SessionId', 'timestamp'])
    return dt.equals(ordered_dt)

print(check_data_sort(data, sample_check=True))
print(check_data_sort(valid))

True
True


In [5]:
## add item index 
### item id를 0번부터 index를 추가합니다.
itemids = data['ItemId'].unique()
n_items = len(itemids)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids).to_dict()
%time data['ItemIdx'] = data['ItemId'].map(lambda x: itemidmap[x])
data[:5]

CPU times: user 1.52 s, sys: 88 ms, total: 1.61 s
Wall time: 1.61 s


,SessionId,ItemId,timestamp,ItemIdx
0,6,214701242,1.396804e+09,0
1,6,214826623,1.396804e+09,1
2,21,214838503,1.396861e+09,2
3,21,214838503,1.396861e+09,2
4,21,214838503,1.396861e+09,2


In [6]:
## offset sessions
### 각 세션의 시작점의 index list를 만든다.
### 즉, 첫번째 sessionid 6의 시작점은 0이고 21의 시작점은 2 이다.
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()
offset_sessions[:5]

array([ 0,  2,  8, 10, 15], dtype=int32)

## Prepare Model

In [7]:
## placeholder & learning rate
X = tf.placeholder(tf.int32, [batch_size], name='input')
Y = tf.placeholder(tf.int32, [batch_size], name='output')
States = [tf.placeholder(tf.float32, [batch_size, rnn_size], name='rnn_state') for _ in range(layers)]
global_step = tf.Variable(0, name='global_step', trainable=False)
lr = tf.maximum(1e-5,tf.train.exponential_decay(
    learning_rate, global_step, decay_steps, decay, staircase=True
)) 

In [8]:
## gru weigths
### input item에 대한 embedding matrix 와
### next item 즉 output을 위한 softmax W, b matrix를 구성한다.
with tf.variable_scope('gru_layer', reuse=tf.AUTO_REUSE):
    #sigma = sigma if sigma != 0 else np.sqrt(6.0 / (n_items + rnn_size))
    #initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)
    initializer = tf.glorot_uniform_initializer()
    embedding = tf.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    softmax_W = tf.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.get_variable('softmax_b', [n_items], initializer=tf.zeros_initializer())

In [9]:
## gru_cell
### ㅁt => ㅁt+1 => ㅁt+2 => ... 
### 위와 같은 recurrent network에서 ㅁ. 즉, 단일 gru cell을 말한다.
with tf.variable_scope('gru_cell', reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.GRUCell(rnn_size, activation=tf.nn.tanh)
    drop_cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=drop_keep_prob)
    stacked_cell = tf.nn.rnn_cell.MultiRNNCell([drop_cell] * layers)

In [10]:
## feedforward gur_cell
### 예를들어 seesion 1의 item sequence가 5, 7, 9 라면,
### 첫번째 배치에서 아이템 5에 대하여 embedding 하여 inputs를 추출하고,
### session 1의 초기 states로 부터 output과 final_state를 계산한다.
### 두번째 배치에서는 아이템 7에 대한 inputs이 들어가고 아이템 5로 부터 계산된
### final state로 아이템 7에 대한 output과 final_state가 다시 계산된다.
### 즉, 배치 순서로 각 seesion의 item sequence가 recurrent하게 학습되는 것이다.
inputs = tf.nn.embedding_lookup(embedding, X)
output, state_ = stacked_cell(inputs, tuple(States))
final_state = state_

In [11]:
## calculate cost(loss)
### is_training(학습)일 경우 negative sampling을 통해 
### cross-entropy loss로 계산하였다. bpt, top1 loss는 주석처리 하였다. 
### ---- negative loss에 대한 추가 설명이 필요함 
### 학습이 아닐 경우 전체 아이템에 대하여 output을 계산한다. 
### --- 예측시 어떻게 사용되는지 추가 설명이 필요함
### for training
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b
yhat = tf.nn.softmax(logits)
### cross-entropy loss
cost = tf.reduce_mean(-tf.log(tf.diag_part(yhat)+1e-24))
### bpr loss
# yhatT = tf.transpose(yhat)
# cost = tf.reduce_mean(-tf.log(tf.nn.sigmoid(tf.diag_part(yhat)-yhatT)))
### top1 loss
# yhatT = tf.transpose(yhat)
# term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat)+yhatT)+tf.nn.sigmoid(yhatT**2), axis=0)
# term2 = tf.nn.sigmoid(tf.diag_part(yhat)**2) / batch_size
# cost = tf.reduce_mean(term1 - term2)
### for prediction
logits_all = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
yhat_all = tf.nn.softmax(logits_all)

In [12]:
## optimize
### Adam optimizer를 사용한다.
optimizer = tf.train.AdamOptimizer(lr)
### grad_cap>0 다면, minimize시 gradient cliping을 수행한다.
### gradient cliping을 수행하는 이유는 다음 블로그 참조 (https://dhhwang89.tistory.com/90)
### 간략하게 학습 중에 gradient가 급격하게 변하는 지점이 발생할 수 있는데, 이는 기존 minima를 찾아가는 방향이 
### 급변할 수 있기 때문에, 이를 방지하기 위해 수행한다.
### 본 학습에서는 cliping을 하지 않는데, 유사하게 learning rate decay을 사용하기 때문인 것으로 생각됨.
tvars = tf.trainable_variables()
gvs = optimizer.compute_gradients(cost, tvars)
if grad_cap > 0:
    capped_gvs = [(tf.clip_by_norm(grad, grad_cap), var) for grad, var in gvs]
else:
    capped_gvs = gvs 
train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

## Training

In [12]:
## session start
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

#### understanding data feed

In [59]:
### 1. 초기 세팅으로 batch_size 만큼 index array를 만들고 maxiter값을 저장한다.
### 2. start는 offset_session(sessionid의 시작 index) 에서 iters를 추출한다.
###    즉, 첫 50개 sessionid의 시작 index를 추출다.
### 3. end는 각 세션에서 다음 세션의 시작되는 index를 추출한다.
iters = np.arange(batch_size)
maxiter = iters.max()
print(iters[:5], "...", iters[-5:])
print(maxiter)
start = offset_sessions[iters]
end = offset_sessions[iters+1]
print(start[:5])
print(end[:5])

[0 1 2 3 4] ... [45 46 47 48 49]
49
[ 0  2  8 10 15]
[ 2  8 10 15 19]


In [60]:
### 1. end - start의 최소 값을 추출한다.
### 만약 최소값이 3라면 즉, 하나의 세션의 item이 두개라면,
### 첫번째 item은 input으로 사용되고 두번째 item은 output으로 사용된다.
### 그리고 다음 배체에서 두번째 item은 input으로 사용되고 세번째 item이 output으로 사용된다.
### 해당 세션은 배치가 두번 돈 후 더이상 학습 할 수 없으므로 다음 세션으로 교체되어야 한다.
### 만약 최소값으 2라면, 해당 세션은 1번 배치 후 다음 세션으로 교체되어야 한다.
### 즉, end - start - 1의 최소 값은 현재 배치된 session의 반복 수를 의미한다.
### 2. out_idx는 각 session의 첫 itemidx를 나타낸다.
minlen = (end-start).min()
out_idx = data.ItemIdx.values[start]
print(minlen)
print(out_idx[:5])

2
[0 2 4 6 7]


In [61]:
### 위에서 설명한 것과 같이, 각 세션의 첫번째 아이템이 in, 두번째 아이템이 out이 된 후 학습에 사용된다.
### minlen - 1 의 수만큼 반복(i)되어 학습한다.
i = 0
in_idx = out_idx
out_idx = data.ItemIdx.values[start+i+1]
print(in_idx[:5])
print(out_idx[:5])

[0 2 4 6 7]
[1 2 5 6 8]


In [62]:
### 1. 위에서 minlen - 1만큼 반복이 완료 되었다는 것은, 교체되어야 할 sessionid가 있다는 말이다.
### start 즉, 세션의 시작 index에서 minlen-1 만큼 증가(반복) 후에 end-start가 1보다 작다는 것은
### 해당 세션의 item이 모두 사용되었다는 것이다. 
### 2. 해당 index를 mask에 저장한다.
### 즉, 배치 index 0(2-1), 2(10-9),... 에 위치한 session의 item은 모두 사용된 것이다.
start = start+minlen-1
mask = np.arange(len(iters))[(end-start)<=1]
print(end[:5])
print(start[:5])
print(mask[:5])

[ 2  8 10 15 19]
[ 1  3  9 11 16]
[ 0  2  8  9 10]


In [63]:
### 1. iters 즉, 배치 index의 0번에 maxiter(49) + 1인 다음 session index로 교체한다.
### 마찬가지고 배치 index 2번에는 그다음 session index 51로 교체한다.
iters[0] = 50
iters[2] = 51
print(iters[:10])
### 2. start와 end는 각 데이터에서 각 세션id의 시작 index와 다음 세션의 시작 index 의미하였다.
### 교체된 session 지점에 새로운 세션id에 대한 시작 index와 다음 세션의 시작 index로 교체한다.
start[0] = offset_sessions[50]
end[0] = offset_sessions[50+1]
start[2] = offset_sessions[51]
end[2] = offset_sessions[51+1]
print(start[:10])
print(end[:10])

[50  1 51  3  4  5  6  7  8  9]
[186   3 188  11  16  20  25  28  33  35]
[188   8 191  15  19  24  27  32  34  36]


In [64]:
### 위 두 단락에서 session item이 다 사용된 iter 0, 2번 index에 대해서만 교체하였는데,
### 모든 교체되어야할 index 즉, 모든 mask에 대하여 session 교체를 수행한다.
for idx in mask:
    maxiter += 1
    if maxiter >= len(offset_sessions)-1:
        finished = True
        break
    iters[idx] = maxiter
    start[idx] = offset_sessions[maxiter]
    end[idx] = offset_sessions[maxiter+1]
print(start[:10])
print(end[:10])

[186   3 188  11  16  20  25  28 191 195]
[188   8 191  15  19  24  27  32 195 198]


In [65]:
### 새로운 session으로 교체되었기 때문에, rnn 학습의 해당 위치의 초기값 state 값도 0으로 초기화해준다.
#if len(mask):
#    for i in range(layers):
#        state[i][mask] = 0

In [66]:
### 전체 과정 (한번의 epoch)을 거치면, 마지막 sessionid 의 index 3140308에서 종료된다.
finished = False
endpoint_count = 0
while not finished:
    minlen = (end-start).min()
    out_idx = data.ItemIdx.values[start]
    for i in range(minlen-1):
        in_idx = out_idx
        out_idx = data.ItemIdx.values[start+i+1]
        endpoint_count += len(out_idx)
    
    start = start+minlen-1
    mask = np.arange(len(iters))[(end-start)<=1]

    for idx in mask:
        maxiter += 1
        if maxiter >= len(offset_sessions)-1:
            finished = True
            break
        iters[idx] = maxiter
        start[idx] = offset_sessions[maxiter]
        end[idx] = offset_sessions[maxiter+1]
        
print(max(start))
data[-5:]

3140308


,SessionId,ItemId,timestamp,ItemIdx
3140305,11562131,214854542,1.411823e+09,21584
3140306,11562151,214536296,1.411769e+09,3483
3140307,11562151,214536296,1.411769e+09,3483
3140308,11562157,214580372,1.411648e+09,1881
3140309,11562157,214516012,1.411648e+09,1880


In [69]:
### SessionId 별 size -1의 합의 전체 학습할 수 있는 데이터인데,
### 위 data feed logic에서는 그보다 적게 끝난다.
### batch size 만큼 session placeholder를 만들고 session의 item이 다 소진되면,
### 다름 세션으로 교체하는 방식인데, 마지막 세션이 교체된 후 minlen 까지만 학습하게 된다.
### 예를들어 마지막 50개 세션에서 49개가 item이 3개가 있고 1개가 2개의 item을 가지고 있다면,
### minlen은 1이 되어 한번 학습 후, 나머지 49개의 세션에는 아직 학습할 item이 남아있지만, 
### 한개의 빈자리에 더이상 교체될 세션이 없어지므로 학습을 종료하게 되기 때문에 enpoint count가 적게 나타난다.
### 예측에서는 위와 조금 다른 방식으로 logic을 바꿔 모든 데이터를 feed할 것이다.
### (학습에서도 모든 데이터를 feed하는 방식으로 바꾸어도 무방할 것으로 보인다.)
print(endpoint_count)
print(sum(data.groupby('SessionId').size() - 1))

2348300
2348603


#### end

In [21]:
## training
### 위 data feeding에서 in_idx, out_idx 후에 실제 학습을 수행하여, 
### epoch만큼 학습을 진행한다.
tic = time.time()
for epoch in range(n_epochs):
    epoch_cost = []
    state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]
    iters = np.arange(batch_size)
    maxiter = iters.max()
    
    start = offset_sessions[iters]
    end = offset_sessions[iters+1]
    
    finished = False
    while not finished:
        minlen = (end-start).min()
        out_idx = data.ItemIdx.values[start]
        for i in range(minlen-1):
            in_idx = out_idx
            out_idx = data.ItemIdx.values[start+i+1]
            # prepare inputs, targeted outputs and hidden states
            fetches = [cost, final_state, global_step, lr, train_op]
            feed_dict = {X: in_idx, Y: out_idx}
            for j in range(layers): 
                feed_dict[States[j]] = state[j]
            
            cost_, state, step, lr_, _ = sess.run(fetches, feed_dict)
            epoch_cost.append(cost_)
                
            if step == 1 or step % print_step == 0:
                avgc = np.mean(epoch_cost)
                print('Epoch {}\tStep {}\tlr: {:.5f}\tloss: {:.4f}\tElapsed: {:.1f}'.
                      format(epoch, step, lr_, avgc, time.time()-tic))

        start = start+minlen-1
        mask = np.arange(len(iters))[(end-start)<=1]
        for idx in mask:
            maxiter += 1
            if maxiter >= len(offset_sessions)-1:
                finished = True
                break
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
        if len(mask):
            for i in range(layers):
                state[i][mask] = 0
        
    avgc = np.mean(epoch_cost)
    if np.isnan(avgc):
        print('Epoch {}: Nan error!'.format(epoch, avgc))
        break
    saver.save(sess, '{}/gru-model'.format(checkpoint_dir), global_step=epoch)
print("1 epoch elapsed time:", time.time() - tic)

Epoch 0	Step 1	lr: 0.00100	loss: 3.9120	Elapsed: 0.2
Epoch 0	Step 1000	lr: 0.00100	loss: 3.3644	Elapsed: 16.0
Epoch 0	Step 2000	lr: 0.00100	loss: 2.9966	Elapsed: 31.8
Epoch 0	Step 3000	lr: 0.00100	loss: 2.8134	Elapsed: 47.7
Epoch 0	Step 4000	lr: 0.00100	loss: 2.6671	Elapsed: 63.7
Epoch 0	Step 5000	lr: 0.00100	loss: 2.5774	Elapsed: 79.4
Epoch 0	Step 6000	lr: 0.00100	loss: 2.4993	Elapsed: 95.2
Epoch 0	Step 7000	lr: 0.00100	loss: 2.4502	Elapsed: 111.0
Epoch 0	Step 8000	lr: 0.00100	loss: 2.3951	Elapsed: 126.8
Epoch 0	Step 9000	lr: 0.00100	loss: 2.3589	Elapsed: 142.6
Epoch 0	Step 10000	lr: 0.00100	loss: 2.3185	Elapsed: 158.4
Epoch 0	Step 11000	lr: 0.00096	loss: 2.2945	Elapsed: 174.1
Epoch 0	Step 12000	lr: 0.00096	loss: 2.2623	Elapsed: 189.8
Epoch 0	Step 13000	lr: 0.00096	loss: 2.2409	Elapsed: 205.6
Epoch 0	Step 14000	lr: 0.00096	loss: 2.2166	Elapsed: 221.3
Epoch 0	Step 15000	lr: 0.00096	loss: 2.1942	Elapsed: 237.0
Epoch 0	Step 16000	lr: 0.00096	loss: 2.1694	Elapsed: 252.7
Epoch 0	Step 17000

Epoch 2	Step 138000	lr: 0.00059	loss: 1.5039	Elapsed: 2180.4
Epoch 2	Step 139000	lr: 0.00059	loss: 1.5044	Elapsed: 2196.3
Epoch 2	Step 140000	lr: 0.00059	loss: 1.5037	Elapsed: 2212.3
1 epoch elapsed time: 2226.7271797657013


In [129]:
sess.close()

## Prediction & Evaluation
valid(test) 데이터에 대하여 예측과 평가를 수행한다.

evaluation metric
1. Recall@20
    - 예측한 top 20 아이템 중에 정답 아이템이 있는지 1 or 0으로 평가 후 전체를 평균함
2. MRR@20 (mean reciprocal rank)
    - 정답 아이템의 rank의 역수를 취한 후 전체를 평균함
    - $mrr = \frac{1}{N} \sum\limits_{i=1}^{N} {\frac{1}{rank_{i}}}$
    - rank가 20위 안에 들지 않으면 0으로 처리한다.

In [13]:
## parameters
cut_off = 20     # @20
batch_size = 50

In [14]:
## session restore
### 마지막(최신) 학습 checkpoint 정보를 restore한다.
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.latest_checkpoint(checkpoint_dir)
saver.restore(sess, ckpt)

INFO:tensorflow:Restoring parameters from ./checkpoint/gru-model-2


In [27]:
## valdation data set
valid['ItemIdx'] = valid['ItemId'].map(lambda x: itemidmap[x])
valid[:5]

,SessionId,ItemId,timestamp,ItemIdx
0,11255868,214853754,1.412011e+09,19930
1,11255868,214577709,1.412011e+09,16686
2,11255868,214853754,1.412011e+09,19930
3,11255882,214855046,1.411965e+09,21684
4,11255882,214854913,1.411965e+09,21665


In [29]:
## valid offset sessions
### 위 학습과 동일하게 각 세션의 시작점의 index list를 만든다.
offset_sessions = np.zeros(valid['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = valid.groupby('SessionId').size().cumsum()
offset_sessions[:5]

array([ 0,  3, 14, 16, 19], dtype=int32)

In [30]:
## init prediction
### 예측 세션의 배치 사이즈 보다 작을 경우 배치 사이즈를 조정한다.
if len(offset_sessions) - 1 < batch_size:
    batch_size = len(offset_sessions) - 1
### training step과 동일
iters = np.arange(batch_size).astype(np.int32)
maxiter = iters.max()
start = offset_sessions[iters]
end = offset_sessions[iters+1]
in_idx = np.zeros(batch_size, dtype=np.int32)
predict_state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]

In [37]:
## prediction & evaluation
### data feeding 요약
### 학습과는 조금 다르게 valid_mask를 설정하여, batch placeholder에 더이상 feed할 세션이 없어지면,
### 해당 위치를 꺼가는 방식으로 모든 batch placeholder가 없어질 때까지 feed하는 것이다.
evalutation_point_count = 0
mrr, recall = 0.0, 0.0
tic = time.time()
while True:
    ### iters는 batch placeholder로 0보다 큰 즉, 마지막 세션까지는 모든 위치를 켜두고
    ### 아래에서 session 데이터가 다 소진되면 해당 위치를 -1로 할당할 것이다.
    ### valid_mask가 0이 되면 즉 모든 위치가 꺼지면 학습을 종료한다.
    valid_mask = iters >= 0
    if valid_mask.sum() == 0:
        print("break at endpoint", evalutation_point_count)
        break
        
    start_valid = start[valid_mask]
    minlen = (end[valid_mask]-start_valid).min()
    in_idx[valid_mask] = valid.ItemIdx.values[start_valid]
    
    for i in range(minlen-1):
        out_idx = valid.ItemIdx.values[start_valid+i+1]
        ## --- prediction --- ##
        fetches = [yhat_all, final_state]
        feed_dict = {X: in_idx}
        for j in range(layers): 
            feed_dict[States[j]] = predict_state[j]
        preds, predict_state = sess.run(fetches, feed_dict)
        preds = pd.DataFrame(data=np.asarray(preds).T)
        preds.fillna(0, inplace=True) ### preds shape: (item_size, batch_size)
        ## --- evaluation --- ##
        in_idx[valid_mask] = out_idx
        ### 정답 아이템 prediction 값보다 높은 아이템이 몇개인지 카운트 하여 rank를 계산한다.
        ranks = (preds.values.T[valid_mask].T > 
                 np.diag(preds.loc[in_idx].values)[valid_mask]).sum(axis=0) + 1
        ### cutoff에 따른 recall과 mrr을 계산한다.
        rank_ok = ranks < cut_off
        recall += rank_ok.sum()
        mrr += (1.0 / ranks[rank_ok]).sum()
        evalutation_point_count += len(ranks)
        
    start = start+minlen-1
    mask = np.arange(len(iters))[(valid_mask) & (end-start<=1)]
    
    for idx in mask:
        maxiter += 1
        ## 더 이상 할당할 세션이 없으면 해당 위치에 -1을 할당하여 끈다.
        if maxiter >= len(offset_sessions)-1:
            iters[idx] = -1
        else:
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
            
    if len(mask):
        for i in range(layers):
            predict_state[i][mask] = 0

### 최종 matric을 계산함.
recall = recall/evalutation_point_count
mrr = mrr/evalutation_point_count
print("recall: ", recall, "mrr:", mrr, "elapsed time:", time.time()-tic)

break at endpoint 5833
recall:  0.4750557174695697 mrr: 0.1688607586708575 elapsed time: 1.7879860401153564


In [70]:
### 모든 데이터가 다 사용되었는지 검증.
print(evalutation_point_count)
print(sum(valid.groupby('SessionId').size() - 1))

5833
5833
